In [ ]:
from pathlib import Path

# Load environment
from dotenv import load_dotenv
load_dotenv("/root/projects/kinsovet/.env")

# Path to JSON files (adjust if needed)
DATA_DIR = Path("/var/lib/kinsovet/data/movies")  # or your actual path

In [ ]:
text = "Когда засуха, пыльные бури и вымирание растений приводят человечество к продовольственному кризису, коллектив исследователей и учёных отправляется сквозь червоточину (которая предположительно соединяет области пространства-времени через большое расстояние) в путешествие, чтобы превзойти прежние ограничения для космических путешествий человека и найти планету с подходящими для человечества условиями."

In [ ]:
prompt = f"""
Перепиши описание фильма своими словами, сохраняя основной сюжет и смысл, но полностью изменяя формулировки. 

Требования:
- Пиши на русском языке
- Делай текст уникальным для коммерческого использования
- Сохраняй ключевые детали: жанр, главных героев, основную завязку
- Избегай дословного копирования фраз
- Делай текст живым и интересным для читателя
- Длина: 3-4 предложения

Исходный текст:
{text}
"""

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
# prompt = "Give me a short introduction to large language model."
prompt = prompt
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)